## The simplest design from Nodes & Springs

We consider, three fully-connected constructions from a minimal number of nodes and springs.

In [4]:
import pyglet, random, math, os, sys, numpy as np, copy
from pyglet.window import key, FPSDisplay
from pyglet import image, shapes

# begin coordinat left down corner
window = pyglet.window.Window(800,280,"Two,three and four spings",resizable = False)
batch = pyglet.graphics.Batch()
label = pyglet.text.Label('', font_name='Arial', bold = True, font_size=20, x=30, y=250, color = (0,0,0,200), batch=batch)
label.text = 'The iteration number is: 0'  # Click the mouse button. 
pyglet.gl.glClearColor(1,1,1,1)

# ===================== Parallel fabers - phrasal logical space =========================== 
N =  9                      # number of bodies
Ns = 8                      # number of springs

pfB  = np.full([N  ,7], np.float64(0)) # phase space of coordinates of bodies [0..6] <=> (m, x,y, dx,dy, ddx,ddy)
pfS  = np.full([Ns ,5], np.float64(0)) # phase-logic state space of springs [0..5] <=> 
                                       # (lr - free distance, k - elasticity coefficient, c - resistance coefficient,
                                       # body1 и body2 - numbers of bodies in the arrays that are connected by the spring)
TS = np.float64(.001)      #time scale
COUNTER = 0                # counter
ON = False                 # running the model

# ===================== programming of the phase-logical space =========================== 

# === 2 NODES & 1 SPINGS ===
#         m,  x,  y,   dx,dy,ddx,ddy
pfB[0] = [10, 40, 30,  0, 0, 0,  0 ]   
pfB[1] = [20, 200,170, 0, 0, 0,  0 ]
#         Lr   k,  c    b1,b2
pfS[0] = [150, 10, 0.3, 0, 1]

# === 3 NODES & 3 SPINGS ===
#         m,  x,  y,  dx,dy,ddx,ddy
pfB[2] = [15, 270,40, 0, 0, 0,  0 ]   
pfB[3] = [20, 370,180,0, 0, 0,  0 ]
pfB[4] = [10, 470,40, 0, 0, 0,  0 ]
#         Lr   k, c    b1,b2 
pfS[1] = [130, 10,0.3, 2, 3]
pfS[2] = [130, 10,0.2, 3, 4]
pfS[3] = [140, 10,0.2, 4, 2]

# === 4 NODES & 4 SPINGS ===
#         m,  x,  y,   dx,dy,ddx,ddy
pfB[5] = [20, 520,60,  0, 0, 0,  0 ]   
pfB[6] = [10, 520,220, 0, 0, 0,  0 ]
pfB[7] = [15, 720,220, 0, 0, 0,  0 ]
pfB[8] = [10, 720,60,  0, 0, 0,  0 ]
#         Lr   k, c    b1,b2 
pfS[4] = [135, 50,0.2, 5, 6]
pfS[5] = [145, 50,0.3, 6, 7]
pfS[6] = [140, 50,0.2, 7, 8]
pfS[7] = [150, 50,0.3, 8, 5]

# ======================= visualization of bodies =============================
Line = [shapes.Line  (0,0, 0,0, color=(100, 100, 100), width=2, batch=batch) for i in range(Ns)]
Body = [shapes.Circle(0,0, 30,  color=(200, 100, 100), batch=batch) for i in range(N)]; # Body[1].color = (0,0,0)

for i in range(N):
    Body[i].radius = pfB[i][0]** (1./3.) * 5         # the size of the mass the root of the third degree
    Body[i].x = pfB[i][1]; Body[i].y = pfB[i][2]     # body coordinates
    
for i in range(Ns):
    b1i = int(pfS[i,3]); b2i = int(pfS[i,4])         # indexes (numbers) of the first and second bodies
    Line[i].x  = pfB[b1i,1]; Line[i].y  = pfB[b1i,2]
    Line[i].x2 = pfB[b2i,1]; Line[i].y2 = pfB[b2i,2]

# ======================= drawing procedure =============================
@window.event
def on_draw():
    window.clear()
    batch.draw()
# ======================= mouse press ============================
@window.event
def on_mouse_press(x, y, button, modifiers):
    global ON
    if button == pyglet.window.mouse.LEFT:
        ON = False if ON else True
# ======================= update procedure ============================
def update(dt):  
    global pfB, pfS, Body, ON, COUNTER, TS, batch
    
    # conjugation of the phase space of bodies with visual representation
    for i in range(N):        
        Body[i].x = pfB[i][1]; Body[i].y = pfB[i][2]
    for i in range(Ns):
        b1i = int(pfS[i,3]); b2i = int(pfS[i,4])  # indexes (numbers) of the first and second bodies
        Line[i].x  = pfB[b1i,1]; Line[i].y  = pfB[b1i,2]
        Line[i].x2 = pfB[b2i,1]; Line[i].y2 = pfB[b2i,2]
    
    if ON:                                                             # excitation
        for j in range(50):
            # calculation of spring-induced accelerations
            for i in range(Ns):
                b1i = int(pfS[i,3]); b2i = int(pfS[i,4])  # indexes (numbers) of the first and second bodies
                lx = pfB[b1i,1]-pfB[b2i,1]          # a = x1-x2
                ly = pfB[b1i,2]-pfB[b2i,2]          # b = y1-y2
                lc = math.sqrt(lx*lx + ly*ly)       # l current
                F = pfS[i,1]*(pfS[i,0]-lc)          # k*(lr-l)
             #ускорение 1 body
                ml1 = pfB[b1i,0]*lc                                        
                pfB[b1i,5] =  F*lx/ml1 - pfB[b1i,3]*pfS[i,2]/pfB[b1i,0]    # ddx1 = (F/m1) * (a/l) - dx*c/m1 
                pfB[b1i,6] =  F*ly/ml1 - pfB[b1i,4]*pfS[i,2]/pfB[b1i,0]    # ddy1 = (F/m1) * (b/l) - dy*c/m2
             #ускорение 2 body          
                ml2 = pfB[b2i,0]*lc                                        
                pfB[b2i,5] = -F*lx/ml2 - pfB[b2i,3]*pfS[i,2]/pfB[b2i,0]    # ddx2 = -(F/m2) * (a/l) - dx*c/m1
                pfB[b2i,6] = -F*ly/ml2 - pfB[b2i,4]*pfS[i,2]/pfB[b2i,0]    # ddy2 = -(F/m2) * (b/l) - dy*c/m2
                                        
             # change of velocity of the 1st body: dx += ddx, dy += ddy taking into account the time scale
                pfB[b1i,3] += pfB[b1i,5]*TS; pfB[b1i,4] += pfB[b1i,6]*TS   
             # change of velocity of the 2st body: dx += ddx, dy += ddy taking into account the time scale
                pfB[b2i,3] += pfB[b2i,5]*TS; pfB[b2i,4] += pfB[b2i,6]*TS   
             
             # change of coordinates of the 1st body: x += dx, y += dy taking into account the time scale
                pfB[b1i,1] += pfB[b1i,3]*TS; pfB[b1i,2] += pfB[b1i,4]*TS    
             # change of coordinates of the 2st body: x += dx, y += dy taking into account the time scale
                pfB[b2i,1] += pfB[b2i,3]*TS; pfB[b2i,2] += pfB[b2i,4]*TS
                           
        COUNTER += 1
        label.text = 'The iteration number is: ' + str(COUNTER)
# =========================== MAIN LOOP ============================
if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/50)
    pyglet.app.run()
    pyglet.clock.unschedule(update)   # necessary to restart the countdown of time intervals
#========================================================================